**6. Exploring book rating data**

In [29]:
liked_books = ["4408", "31147619", "29983711", "9401317", "9317691", "8153988", "20494944"]

**Find all the users that like the same books as us then find all the books that they liked (Assumption: They have similar taste).**

In [30]:
!head book_id_map.csv

500 Can't connect to book_id_map.csv:80 (Bad hostname 'book_id_map.csv')
Content-Type: text/plain
Client-Date: Tue, 04 Apr 2023 08:17:42 GMT
Client-Warning: Internal response



In [31]:
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.split(",")
        csv_book_mapping[csv_id] = book_id

In [32]:
len(csv_book_mapping)

2360651

In [33]:
!wc -l goodreads_interactions.csv

 228648343 goodreads_interactions.csv


**Size of goodreads_interactions.csv**

In [34]:
!ls -lh | grep goodreads_interactions.csv

-rw-r--r--@ 1 britneyagius  staff   4.0G Mar  8 15:47 goodreads_interactions.csv


**7. Finding users who like the same books as us**

**A set is a python data structure where every element is unique.**

**if user_id in overlap_users: continue - means that if we already added this user to our overlap user set we don't need to keep processing.**

In [35]:
!head goodreads_interactions.csv

500 Can't connect to goodreads_interactions.csv:80 (Bad hostname 'goodreads_interactions.csv')
Content-Type: text/plain
Client-Date: Tue, 04 Apr 2023 08:18:24 GMT
Client-Warning: Internal response



In [36]:
overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue

        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id)

**8. Finding what books those users liked**

**rec_lines will only contin books that users who likes the same books as us have read.**

**rec_lines will contain books we might want to read.**

In [37]:
rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [38]:
len(overlap_users)

0

In [43]:
len(rec_lines)

0

In [39]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

**top_recs = recs("book_id").value_counts() - It counts up how many times each book id occured and it shows you the most common ones so our top recommendations will be our book ids that occured the most frequently.**

In [40]:
top_recs = recs["book_id"].value_counts().head(10)

**To get from the book id to a title: Read in from the file books_titles.json**

In [41]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [42]:
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook
